In [27]:
from preprocess import *

In [87]:
data, labels = load_data('THUCNews-Title')

In [88]:
len(data),len(labels)

(836075, 836075)

In [30]:
wvm = create_vector(data, labels, 64)
pickle_dump('wvm', wvm)

In [89]:
train_data, test_data, train_labels, test_labels = split_train_test_data(data, labels, test_size=0.3)

In [86]:
spider_test_data, spider_test_labels = test_data, test_labels

In [90]:
labelset = list(set(train_labels))
idx2label = dict(enumerate(labelset))
label2idx = dict([(v, k) for k, v in enumerate(labelset)])

train_data_id = [text_to_id(text, wvm, 30) for text in train_data ]
train_labels_idx = np.array([label2idx[label] for label in train_labels])


test_data_id = [text_to_id(text, wvm, 30) for text in test_data ]
test_labels_idx = np.array([label2idx[label] for label in test_labels])

In [33]:
embedding_matrix = np.zeros((len(wvm.wv.index_to_key), wvm.vector_size))
for i, word in enumerate(wvm.wv.index_to_key):
    embedding_matrix[i] = wvm.wv[word]

In [34]:
pickle_dump('idx2label', idx2label)
pickle_dump('label2idx', label2idx)
pickle_dump('wvm', wvm)
pickle_dump('embedding_matrix', embedding_matrix)

In [35]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_classes):
        super(BiLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=True)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        avg_pool = torch.mean(lstm_out, dim=1)
        output = self.fc(avg_pool)
        return output

In [36]:


hidden_dim = 64
num_classes = len(label2idx)
model = BiLSTMClassifier(embedding_matrix, hidden_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [77]:
train_data_id_tensor = torch.LongTensor(train_data_id)
train_labels_idx_tensor = torch.LongTensor(train_labels_idx)

test_data_id_tensor = torch.LongTensor(test_data_id)
test_labels_idx_tensor = torch.LongTensor(test_labels_idx)

In [78]:
model = model.to('cuda')

train_data_id_tensor = train_data_id_tensor.to('cuda')
train_labels_idx_tensor = train_labels_idx_tensor.to('cuda')


test_data_id_tensor = test_data_id_tensor.to('cuda')
test_labels_idx_tensor = test_labels_idx_tensor.to('cuda')

In [39]:
batch_size = 128

train_dataset = torch.utils.data.TensorDataset(train_data_id_tensor, train_labels_idx_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [40]:
model.train()
num_epochs = 5
model.to('cuda')
epoch_loss = []
for epoch in range(num_epochs):
    total_loss = 0.0
    i = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        i+=1
        if(i % 1000 == 0):
            print('\r batch: {} loss: {}'.format(i, loss.item()))
    
    average_loss = total_loss / len(train_loader)
    fpath = os.path.join('pth',f'epoch_{epoch}.pth')
    torch.save(model,fpath)
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {average_loss:.4f}')
    epoch_loss.append(average_loss)

 batch: 1000 loss: 0.6460931301116943
 batch: 2000 loss: 0.42555859684944153
 batch: 3000 loss: 0.33056896924972534
 batch: 4000 loss: 0.29724764823913574
Epoch [1/5] Loss: 0.5234
 batch: 1000 loss: 0.4113433063030243
 batch: 2000 loss: 0.362537682056427
 batch: 3000 loss: 0.29793497920036316
 batch: 4000 loss: 0.28559476137161255
Epoch [2/5] Loss: 0.3502
 batch: 1000 loss: 0.3567611575126648
 batch: 2000 loss: 0.37557128071784973
 batch: 3000 loss: 0.2742232382297516
 batch: 4000 loss: 0.2797883450984955
Epoch [3/5] Loss: 0.3126
 batch: 1000 loss: 0.3430483043193817
 batch: 2000 loss: 0.32845520973205566
 batch: 3000 loss: 0.25019142031669617
 batch: 4000 loss: 0.27296218276023865
Epoch [4/5] Loss: 0.2908
 batch: 1000 loss: 0.3329230546951294
 batch: 2000 loss: 0.2826218008995056
 batch: 3000 loss: 0.23252573609352112
 batch: 4000 loss: 0.26305222511291504
Epoch [5/5] Loss: 0.2760


In [70]:
model.eval()
a = 0
b = 200
with torch.no_grad():
    test_outputs = model(test_data_id_tensor[a:b])
    _, predicted = torch.max(test_outputs, 1)
    predicted = predicted.to('cpu')
    accuracy = accuracy_score(test_labels_idx_tensor.to('cpu')[a:b], predicted.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 91.50%


In [91]:
print(idx2label)

{0: '社会', 1: '科技', 2: '娱乐', 3: '时政', 4: '房产', 5: '教育', 6: '时尚', 7: '体育', 8: '财经', 9: '彩票', 10: '星座', 11: '家居', 12: '游戏', 13: '股票'}


In [80]:
model.eval()
a = 0
b = 200
with torch.no_grad():
    test_outputs = model(test_data_id_tensor[a:b])
    _, predicted = torch.max(test_outputs, 1)
    predicted = predicted.to('cpu')
    accuracy = accuracy_score(test_labels_idx_tensor.to('cpu')[a:b], predicted.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 5.00%


In [95]:
def predict(text):
    with torch.no_grad():
        test_outputs = model(torch.LongTensor([text_to_id(text, wvm, 30)]).to('cuda'))
        _, predicted = torch.max(test_outputs, 1)
        predicted = predicted.to('cpu').numpy()
        predicted = idx2label[predicted[0]]
    return predicted

In [98]:
spider_test_data[:100]

['安徽启动防汛防台风四级应急响应',
 '“老头乐”，究竟该去还是留？',
 '天津百利机械装备集团有限公司原党委书记、董事长王东江严重违纪违法被开除党籍和公职',
 '病人自述被医生遗忘在磁共振舱3小时！医院称涉事医生已处罚，当地卫健局介入调查',
 '广州消防：肇花高速一辆货车起火 现场明火扑灭 无人员伤亡',
 '为抢救生命不计代价 产妇大出血 医护人员奋战20多小时供血6.75万毫升',
 '我国新型液氧甲烷火箭朱雀二号发射成功',
 '黑龙江省人大教育科学文化卫生委员会原副主任委员朱志强接受审查调查',
 '安徽聚力推动制造业高质量发展（高质量发展调研行）',
 '这样做，让老年人远离多种慢性病的困扰',
 '“同人作品第一案”二审宣判 作家江南赔金庸方188万',
 '江西警方捣毁“代孕”诈骗团伙：受害人涉及10余省，金额高达几百万',
 '辽宁盘锦“1·15”重大生产安全责任事故调查报告公布',
 '中央纪委国家监委第二监督检查室副主任、一级巡视员刘燃被开除党籍和公职',
 '假低价、真诱导！网购需警惕电商平台“价格坑”',
 '全国职工互助保障参保人数达5226万人次',
 '云南省地震局：滇池白鲢鱼跳龙门初步分析受水体和人为影响',
 '北京海关今年已截获进境“异宠”128只',
 '【8点见】男子掏警官证让交警放行？官方通报',
 '贵州“6·18梵净山金顶摩崖刻字案”违法行为人已被锁定',
 '重庆警方破获一特大虚开增值税专用发票案 涉案金额超20亿元',
 '古琴名家齐聚上海 奏响千年非遗古韵',
 '湖南回应涉“省直单位”微信群不雅信息事件：纪检部门第一时间介入 警方正在调查',
 '部分交友软件仍对未成年人“不设防”',
 '天宫TV | 点赞！神十六出舱任务精彩回顾',
 '中国第13次北冰洋科学考察首个海域作业任务全部完成',
 '公司为竞标将5万P成5000万 法院：顶格罚款！',
 '不给“踢皮球”留下空间 深圳打造民生诉求服务一体化平台',
 '“求爱不成当街捅杀女子”案检方建议死刑，凶手当庭忏悔但未道歉',
 '抓好关键期 各方助力高校毕业生就业',
 '就在2024年！AG600有新计划',
 '答疑！甲流高发，公众如何科学应对？',
 '重庆文化产业投资集团有限公司党委书记、董事长陈余莉接受审查调查',
 '电信网络诈骗

In [102]:
predicted = [predict(d) for d in test_data[:100]]
for t, p in zip(test_data, predicted):
    print(p,'\t' ,t)

时政 	 安徽启动防汛防台风四级应急响应
教育 	 “老头乐”，究竟该去还是留？
体育 	 天津百利机械装备集团有限公司原党委书记、董事长王东江严重违纪违法被开除党籍和公职
体育 	 病人自述被医生遗忘在磁共振舱3小时！医院称涉事医生已处罚，当地卫健局介入调查
体育 	 广州消防：肇花高速一辆货车起火 现场明火扑灭 无人员伤亡
体育 	 为抢救生命不计代价 产妇大出血 医护人员奋战20多小时供血6.75万毫升
科技 	 我国新型液氧甲烷火箭朱雀二号发射成功
体育 	 黑龙江省人大教育科学文化卫生委员会原副主任委员朱志强接受审查调查
股票 	 安徽聚力推动制造业高质量发展（高质量发展调研行）
时尚 	 这样做，让老年人远离多种慢性病的困扰
体育 	 “同人作品第一案”二审宣判 作家江南赔金庸方188万
体育 	 江西警方捣毁“代孕”诈骗团伙：受害人涉及10余省，金额高达几百万
体育 	 辽宁盘锦“1·15”重大生产安全责任事故调查报告公布
体育 	 中央纪委国家监委第二监督检查室副主任、一级巡视员刘燃被开除党籍和公职
科技 	 假低价、真诱导！网购需警惕电商平台“价格坑”
时政 	 全国职工互助保障参保人数达5226万人次
体育 	 云南省地震局：滇池白鲢鱼跳龙门初步分析受水体和人为影响
教育 	 北京海关今年已截获进境“异宠”128只
社会 	 【8点见】男子掏警官证让交警放行？官方通报
体育 	 贵州“6·18梵净山金顶摩崖刻字案”违法行为人已被锁定
体育 	 重庆警方破获一特大虚开增值税专用发票案 涉案金额超20亿元
娱乐 	 古琴名家齐聚上海 奏响千年非遗古韵
体育 	 湖南回应涉“省直单位”微信群不雅信息事件：纪检部门第一时间介入 警方正在调查
教育 	 部分交友软件仍对未成年人“不设防”
娱乐 	 天宫TV | 点赞！神十六出舱任务精彩回顾
体育 	 中国第13次北冰洋科学考察首个海域作业任务全部完成
社会 	 公司为竞标将5万P成5000万 法院：顶格罚款！
体育 	 不给“踢皮球”留下空间 深圳打造民生诉求服务一体化平台
体育 	 “求爱不成当街捅杀女子”案检方建议死刑，凶手当庭忏悔但未道歉
教育 	 抓好关键期 各方助力高校毕业生就业
科技 	 就在2024年！AG600有新计划
教育 	 答疑！甲流高发，公众如何科学应对？
股票 	 重庆文化产业投